# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **ML Model - Predict Remaining Useful Life (RUL)**

## Objectives

Answer [Business Requirement 1](https://github.com/roeszler/filter-maintenance-predictor/blob/main/README.md#business-requirements) :
*   Fit and evaluate a **regression model** to predict the Remaining Useful Life of a replaceable part
*   Fit and evaluate a **classification model** to predict the Remaining Useful Life of a replaceable part should the regressor not perform well.

## Inputs

Data cleaning and feature engineering from their respective notebooks:
* inputs/datasets/cleaned/df_train.csv
* inputs/datasets/cleaned/df_test.csv
* inputs/datasets/cleaned/df_validate.csv

## Outputs

* Train set (features and target)
* Test set (features and target)
* Validation set (features and target)
* ML pipeline to predict RUL
* A map of the labels
* Feature Importance Plot



---

### The major steps in a Regressor Pipeline

1. **ML Pipeline: Regressor**
    * Create Regressor Pipeline
    * Split the train set
    * Grid Search CV SKLearn
        * Use standard hyperparameters to find most suitable algorithm
        * Extensive search on most suitable algorithm to find the best hyperparameter configuration
    * Assess Feature Performance
    * Evaluate Regressor
    * Create Train, Test, Validation Sets

2. **ML Pipeline: Regressor + Principal Component Analysis (PCA)**
    * Prepare the Data for the Pipeline
    * Create Regressor + PCA Pipeline
    * Split the train and validation sets
    * Grid Search CV SKLearn
        * Use standard hyperparameters to find most suitable algorithm
        * Do an extensive search on most suitable algorithm to find the best hyperparameter configuration
    * Assess Feature Performance
    * Evaluate Regressor
    * Create Train, Test, Validation Sets

_Optionally_

3. **Convert Regression to Classification**
    * Convert numerical target to bins, and check if it is balanced
    * Rewrite Pipeline for ML Modelling
    * Load Algorithms For Classification
    * Split the Train Test sets:
    * Grid Search CV SKLearn:
        * Use standard hyper parameters to find most suitable model
        * Grid Search CV
        * Check Result
    * Do an extensive search on the most suitable model to find the best hyperparameter configuration.
        * Define Model Parameters
        * Extensive Grid Search CV                             
        * Check Results
        * Check Best Model
        * Parameters for best model
        * Define the best clf_pipeline
    * Assess Feature Importance
    * Evaluate Classifier on Train and Test Sets
        * Custom Function
        * List that relates the classes and tenure interval

4. **Decide which pipeline to use**

5. **Refit with the best features**
    * Rewrite Pipeline
    * Split Train Test Set with only best features
    * Subset best features
    * Grid Search CV SKLearn
    * Best Parameters
        * Manually
    * Grid Search CV
    * Check Results
    * Check Best Model
    * Define the best pipeline

6. **Assess Feature Importance**

7. **Push Files to Repo**

<!-- Modelling:
The hypothesis part of the process where you will find out whether you can answer the question.
* Identify what techniques to use.
* Split your data into train, validate and test sets.
* Build and train the models with the train data set.
* Validate Models and hyper-parameter : Trial different machine learning methods and models with the validation data set.
* Poor Results - return to data preparation for feature engineering
* Successful hypothesis - where the inputs from the data set are mapped to the output target / label appropriately to evaluate.

5. Evaluation:
Where you test whether the model can predict unseen data.
* Test Dataset
* Choose the model that meets the business success criteria best.
* Review and document the work that you have done.
* If your project meets the success metrics you defined with your customer?
- Ready to deploy. -->

---

In [ ]:
df_stop